In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import operator
import tensorflow as tf
import random

In [ ]:
DIRECTORY = r'C:\Users\DIPTO\Desktop\train'
VAL_DIRECTORY = r'C:\Users\DIPTO\Desktop\val'

CATEGORIES = ['all', 'hem']

In [ ]:
def crop_center(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [ ]:
data = []
i = 0
plt.figure(figsize=(15, 15))
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path)
        crop_arr = crop_center(arr, (210,210))
        if 1 <= i+1 <= 140:                      # total 140 image
            ax = plt.subplot(13, 11, i+1)
        plt.imshow(crop_arr)
        i += 1
        data.append([crop_arr, label])

In [ ]:
random.shuffle(data)

x_train = []
y_train = []

for features, label in data:
    x_train.append(features)
    y_train.append(label)

    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
val_data = []
i = 0
plt.figure(figsize=(15, 15))
for category in CATEGORIES:
    path = os.path.join(VAL_DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path)
        crop_arr = crop_center(arr, (210,210))
        if 1 <= i+1 <= 70:                     # total image 70
            ax = plt.subplot(10, 7, i+1)
        plt.imshow(crop_arr)
        i += 1
        val_data.append([crop_arr, label])

In [ ]:
random.shuffle(val_data)

x_val = []
y_val = []

for features, label in val_data:
    x_val.append(features)
    y_val.append(label)

    
x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
x_train = x_train/255
x_val   = x_val/255

In [ ]:
model = tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(210,210,3),
    pooling=None,
    classes=1,
    classifier_activation="sigmoid",
)

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam, RMSprop
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5)
#sgd_opt = SGD(lr=1e-06, momentum=0.0, decay=0.0, nesterov=False)
rmsp_opt = RMSprop(lr=1e-4, decay=0.9)
# eve_opt = Eve(lr=1e-4, decay=1E-4, beta_1=0.9, beta_2=0.999, beta_3=0.999, small_k=0.1, big_K=10, epsilon=1e-08)

model.compile(optimizer= adam_opt,
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
import keras
callbacks = [
    keras.callbacks.ModelCheckpoint('baseline_vgg.h5', monitor='val_accuracy', save_best_only=True, mode='max'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, verbose=1, patience=5, mode='max')]


In [ ]:
model.fit(x_train, y_train, validation_data = (x_val,y_val), verbose = 1, batch_size=28, epochs = 1, shuffle = True, callbacks = callbacks)